# Healthcare Marketing Analytics Project
## Determining the impact of R&D Ration on Healthcare Firms' Value

In [2]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
warnings.simplefilter("once")

### Data Sourcing
This dataset was provided by Bocconi University, Marketing Analytics department. Here is the description of the data columns in the dataset:
- Firm ID (variable name: id)
- Year (variable name: year)
- Firm Total Assets (unit: million $) (variable name: assets)
- Long-term Debt (unit: million $) (variable name: debt)
- Earnings (unit: million $): This variable is EBITA (earnings before interest, taxes, depreciation,
and amortization). (variable name: earnings)
- Cash Flows (unit: million $): This is Net Cash Flow from Operating Activity. However, please
conceptually consider this variable a proxy for profit based on only cash as some academic
studies do. (variable name: cashflow)
- Sales (unit: million $) (variable name: sales)
- Advertising Expenditure (unit: million $): Conceptually, you can consider this variable as a firm’s
brand building effort. (variable name: ad)
- R&D Expenditure (unit: million $): Conceptually, you can consider this variable as a firm’s
innovation effort. (variable name: rd)
- Marketing, Selling, and Operating Expenditure (unit: million $): Conceptually, you can consider
this variable as a firm’s selling and marketing effort (e.g., sale forces, customer management)
except advertising. Note that this variable does not include advertising expenditure and R&D
expenditure. (variable name: mkt)
- Product Market Threat: Consider this variable as competitive threats that a firm goes through in
its product markets. Higher values indicate higher levels of competitive threats. (variable name:
threat)
- 2001 Recession (dummy): 1 if the observation is in the year of 2001 recession and 0 otherwise.
(variable name: rec1)
- 2008 Recession (dummy): 1 if the observation is in the year of 2008 recession and 0 otherwise.
(variable name: rec2)
- Firm Value (i.e., Tobin’s Q) (variable name: fv)
- Market Valuation (unit: million $): Market value of a firm (variable name: mv)
- Business sector where a firm operates (indicator) (variable name: sector)
    - 1: Manufacturing (Consumer Durables and Non-Durables) e.g., Food, Tobacco, Textile, Apparel, Toys, Cars, TVs, Furniture, Appliances
    - 2: Manufacturing (Non-Consumer Goods) e.g., Machinery, Office Furniture, Paper, Computers, Electronic Equipment
    - 3: Wholesale
    - 4: Retailing
    - 5: Healthcare e.g., healthcare, medical equipment, drugs
    - 6: Information Technology e.g., software publishers, telecommunications, web search portals
    - 7: Service Sector (Technical Services, Professional and Administrative Services, and Education Services)
    - 8: Entertainment, Accommodation, and Full Service Restaurants

In [4]:
all_data = pd.read_csv('marketing_insight.csv')
all_data.head()

,id,year,assets,debt,earnings,cashflow,sales,mv,ad,rd,threat,fv,rec1,rec2,mkt,sector
0,42,1993,4.103,0.000,-1.748,-2.175,3.610,63.674000,0.074,0.305,1.7482,14.836461,0,0,2.042,5
1,2607,1993,18.897,0.716,-2.458,-2.016,26.274,10.802283,0.748,1.310,18.0228,0.083362,0,0,5.405,1
2,1204,1993,37.476,23.735,4.359,-0.087,63.938,29.856251,0.759,0.542,1.0184,1.225564,0,0,9.997,3
3,899,1993,7.882,0.011,-4.933,-4.958,5.411,58.403999,0.775,2.059,9.1024,6.930602,0,0,6.089,5
4,1746,1993,24.496,0.000,7.723,-5.843,43.030,208.065000,0.719,0.372,1.0658,7.952849,0,0,9.994,5


In [7]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22723 entries, 0 to 22722
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        22723 non-null  int64  
 1   year      22723 non-null  int64  
 2   assets    22723 non-null  float64
 3   debt      22723 non-null  float64
 4   earnings  22723 non-null  float64
 5   cashflow  22723 non-null  float64
 6   sales     22723 non-null  float64
 7   mv        22723 non-null  float64
 8   ad        22723 non-null  float64
 9   rd        22723 non-null  float64
 10  threat    22723 non-null  float64
 11  fv        22723 non-null  float64
 12  rec1      22723 non-null  int64  
 13  rec2      22723 non-null  int64  
 14  mkt       22723 non-null  float64
 15  sector    22723 non-null  int64  
dtypes: float64(11), int64(5)
memory usage: 2.8 MB


#### Filtering out the Healthcare Sector
This project will be focused on the Healthcare sector, therefore, we'll filter out other sectors to remain with **sector 5** – ***Healthcare***.

In [9]:
data = all_data[all_data['sector']==5] #Filtering out all sectors to remain with healthcare data
data.head()

,id,year,assets,debt,earnings,cashflow,sales,mv,ad,rd,threat,fv,rec1,rec2,mkt,sector
0,42,1993,4.103,0.000,-1.748,-2.175,3.610,63.674000,0.074,0.305,1.7482,14.836461,0,0,2.042,5
3,899,1993,7.882,0.011,-4.933,-4.958,5.411,58.403999,0.775,2.059,9.1024,6.930602,0,0,6.089,5
4,1746,1993,24.496,0.000,7.723,-5.843,43.030,208.065000,0.719,0.372,1.0658,7.952849,0,0,9.994,5
10,1312,1993,10.248,0.000,-15.562,-9.349,0.000,23.978001,0.000,4.075,5.0967,2.646077,0,0,11.487,5
19,1686,1993,6.007,0.929,1.055,0.483,6.980,19.942678,0.086,0.197,1.0549,3.027747,0,0,1.558,5


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3292 entries, 0 to 22569
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        3292 non-null   int64  
 1   year      3292 non-null   int64  
 2   assets    3292 non-null   float64
 3   debt      3292 non-null   float64
 4   earnings  3292 non-null   float64
 5   cashflow  3292 non-null   float64
 6   sales     3292 non-null   float64
 7   mv        3292 non-null   float64
 8   ad        3292 non-null   float64
 9   rd        3292 non-null   float64
 10  threat    3292 non-null   float64
 11  fv        3292 non-null   float64
 12  rec1      3292 non-null   int64  
 13  rec2      3292 non-null   int64  
 14  mkt       3292 non-null   float64
 15  sector    3292 non-null   int64  
dtypes: float64(11), int64(5)
memory usage: 437.2 KB


In [11]:
data.describe()

,id,year,assets,debt,earnings,cashflow,sales,mv,ad,rd,threat,fv,rec1,rec2,mkt,sector
count,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.000000,3292.0
mean,1724.311665,2007.315614,3429.609149,775.250014,563.978191,418.321345,1706.000846,6780.085848,71.307075,255.238051,8.154948,2.360336,0.034933,0.045261,408.670110,5.0
std,1058.535967,7.179453,16334.438091,3832.730215,2663.596820,2038.262931,7211.308404,29286.544040,354.174284,1154.296713,11.103680,2.746864,0.183638,0.207908,1715.608140,0.0
min,1.000000,1993.000000,0.404000,0.000000,-666.676000,-635.442000,0.000000,0.809150,0.000000,0.000000,1.000000,-0.586277,0.000000,0.000000,-1895.591100,5.0
25%,774.000000,2002.000000,34.815250,0.000000,-5.244250,-4.947750,26.508250,52.787131,0.288000,1.748750,1.294500,0.861938,0.000000,0.000000,12.847250,5.0
50%,1584.000000,2007.000000,109.109000,1.957500,4.940000,3.104500,84.768000,207.849600,1.103000,6.816000,3.197450,1.604317,0.000000,0.000000,34.998501,5.0
75%,2629.000000,2013.000000,495.226000,65.151500,58.128250,39.787500,354.252500,1014.843800,6.500000,30.600500,10.694725,2.901638,0.000000,0.000000,119.889250,5.0
max,3523.000000,2019.000000,212949.000000,63226.000000,28795.000000,23416.000000,82059.000000,384003.780000,4000.000000,12245.000000,85.363701,45.089115,1.000000,1.000000,19088.000000,5.0
